In [45]:
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from pyxirr import xirr
pd.options.display.float_format = '{:,.2f}'.format


# Imputs

In [46]:
df_input=pd.read_csv('./input_fluxo_caixa_SQS312h604.csv',header=None,index_col=0)
df_input=df_input.rename(columns={1:'valor_variavel'})

In [47]:
valor_imovel=df_input.loc['valor_imovel','valor_variavel']
entrada_pp=df_input.loc['entrada_pp','valor_variavel']
entrada_por_valor=df_input.loc['entrada_por_valor','valor_variavel']
financiamento_0_ou_1=df_input.loc['financiamento_0_ou_1','valor_variavel']
area=df_input.loc['area','valor_variavel']

# Custos Iniciais

In [48]:
itbi_parcelado_0_ou_1=df_input.loc['itbi_parcelado_0_ou_1','valor_variavel']
custo_itbi=0.03*valor_imovel
custo_escritura=df_input.loc['custo_escritura','valor_variavel']
custo_registro=df_input.loc['custo_registro','valor_variavel']


# Cálculo financiamento

In [49]:
# Cálculo do juros em função do mês
def parcela(n):
    saldo_devedor=valor_finaciado-(n-1)*amortizacao
    juros=saldo_devedor*juros_nominal/12
    return amortizacao+juros+encargos_finaciamento


In [50]:
if financiamento_0_ou_1==1:
    n_parcelas=int(df_input.loc['n_parcelas','valor_variavel'])                         # 35 anos
    df=pd.DataFrame(columns=['parcela','saldo_devedor'],index=range(0,n_parcelas))
    juros_nominal=df_input.loc['juros_nominal','valor_variavel']                        # a.a    
    juros_tr=df_input.loc['tr','valor_variavel']                        # a.a
    juros_nominal=juros_nominal+juros_tr
    custo_avaliacao=df_input.loc['custo_avaliacao','valor_variavel']
    juros_efetivo=((1+juros_nominal/12)**(12)-1)            # Efeito anual dos juros sobre juros mensais.  

    # he usual way to test for a NaN is to see if it's equal to itself:
    if entrada_por_valor!=entrada_por_valor:
        # Não digitou o valor da entrada
        if entrada_pp!=entrada_pp:
            print('Digite o valor da entrada ou seu percentual.')
        else:
            valor_entrada=valor_imovel*entrada_pp
    else:
        valor_entrada=entrada_por_valor
    
    valor_finaciado=valor_imovel-valor_entrada
    encargos_finaciamento=df_input.loc['encargos_finaciamento','valor_variavel']
    amortizacao=valor_finaciado/n_parcelas
    saldo_devedor=valor_finaciado

else:
    amortizacao=0
    saldo_devedor=0
    n_parcelas=0   
    juros_nominal=0
    custo_avaliacao=0
    juros_efetivo=0  
    valor_finaciado=0
    encargos_finaciamento=0
    valor_entrada=valor_imovel
    df=pd.DataFrame()

In [51]:
total_custos_iniciais=valor_entrada+custo_escritura+custo_registro+custo_avaliacao

In [52]:
for i in range(0,max(24,n_parcelas+1)):             # Se não for financiado seta o df para 24 linhas inicialmente
    if i==0:
        df.loc[i,'saldo_devedor']=valor_finaciado-i*amortizacao
        df.loc[i,'parcela']=0
    else:

        df.loc[i,'saldo_devedor']=valor_finaciado-i*amortizacao
        df.loc[i,'parcela']=parcela(i)

In [53]:
# ITBI
if itbi_parcelado_0_ou_1==1:
    for i in range(0,len(df)):
        if i <10:
            df.loc[i,'itbi']=custo_itbi/10
        else:
            df.loc[i,'itbi']=0
else:
    df.loc[0,'itbi']=custo_itbi
    for i in range(1,len(df)):
        df.loc[0,'itbi']=0

# Custos fixos

In [54]:
condominio=df_input.loc['condominio','valor_variavel']
iptu_am=df_input.loc['iptu_aa','valor_variavel']/12
luz=df_input.loc['luz','valor_variavel']

total_custos_fixos=condominio+iptu_am+luz

In [55]:
for i in range(0,len(df)):
    df.loc[i,'custos_fixos']=total_custos_fixos
    if i==0:
        df.loc[i,'custos_iniciais']=total_custos_iniciais
    else:
        df.loc[i,'custos_iniciais']=0

# Custos com reforma

In [56]:
obra=df_input.loc['obra','valor_variavel']
prazo_obra=df_input.loc['prazo_obra','valor_variavel']                # Prazo da obra em mesês

def reforma(i, n,total):
    if i==0:
        return 0
    elif i<=n:
        return total/n
    else:
        return 0

for i in range(0,len(df)):
    df.loc[i,'reforma']=reforma(i,prazo_obra,obra)

# Cenários de venda

In [57]:
valor_venda=df_input.loc['valor_venda','valor_variavel']
prazo_venda=int(prazo_obra+df_input.loc['prazo_venda_pos_obra','valor_variavel'])                # Prazo da obra em mesês + venda
comissao=df_input.loc['comissao','valor_variavel']

In [58]:
def venda(i, n,total):
    if i==n:
        print(f'Comissão: {comissao*valor_venda}')
        return -total*(1-comissao)
    else:
        return 0

for i in range(0,len(df)):
    df.loc[i,'venda']=venda(i,prazo_venda,valor_venda)
df['total']=df.sum(axis=1)
df['total']=df['total']-df['saldo_devedor']

# quitação do saldo devedor e anulação das despesas à partir da venda
for i in range(0,len(df)):
    if i>prazo_venda:
        df.loc[i,'total']=0
    elif i==prazo_venda:
        df.loc[i,'total']=df.loc[i,'total']+df.loc[i,'saldo_devedor']

Comissão: 12900.0


In [59]:
delta = relativedelta(months=1)
delta_d = timedelta(days=1)
start_date = datetime.today().date()+15*delta_d

for i in range(0, len(df)):
    if i==0:
        df.loc[i,'data']=start_date
    else:
        df.loc[i,'data']=start_date+i*delta

# Filtra apenas até a célula de venda
df=df[df['total']!=0]
df

,parcela,saldo_devedor,itbi,custos_fixos,custos_iniciais,reforma,venda,total,data
0,0,"560,000.00","2,130.00",885.67,"154,000.00",0.00,0.00,"157,015.67",2022-09-22
1,"6,265.33","558,666.67","2,130.00",885.67,0.00,"12,500.00",0.00,"21,781.00",2022-10-22
2,"6,253.90","557,333.33","2,130.00",885.67,0.00,"12,500.00",0.00,"21,769.57",2022-11-22
3,"6,242.47","556,000.00","2,130.00",885.67,0.00,"12,500.00",0.00,"21,758.13",2022-12-22
4,"6,231.03","554,666.67","2,130.00",885.67,0.00,"12,500.00",0.00,"21,746.70",2023-01-22
5,"6,219.60","553,333.33","2,130.00",885.67,0.00,0.00,0.00,"9,235.27",2023-02-22
6,"6,208.17","552,000.00","2,130.00",885.67,0.00,0.00,0.00,"9,223.83",2023-03-22
7,"6,196.73","550,666.67","2,130.00",885.67,0.00,0.00,0.00,"9,212.40",2023-04-22
8,"6,185.30","549,333.33","2,130.00",885.67,0.00,0.00,"-847,100.00","-288,565.70",2023-05-22


In [60]:
# Importa output da BcB_forcast (os dois diretórios no mesmo caminho)
df_juros_real=pd.read_csv(r'../BcB_Forcast/output/num_indice_selic_ipca_diferenca.csv', index_col=0)
df_juros_real

,num_indice_selic,num_indice_ipca,dif_num_indice
2022-09-07,100.00,100.13,99.87
2022-09-08,100.04,100.14,99.89
2022-09-09,100.07,100.15,99.92
2022-09-10,100.11,100.15,99.95
2022-09-11,100.14,100.16,99.98
...,...,...,...
2024-08-10,124.78,109.77,115.01
2024-08-11,124.81,109.77,115.04
2024-08-12,124.84,109.78,115.06
2024-08-13,124.87,109.79,115.09


In [61]:
def busca_num_indice_selic(data):
    #converte data em string
    data=datetime.strftime(data,'%Y-%m-%d')    
    return df_juros_real.loc[data,'num_indice_selic']
df['num_selic']=df.apply(lambda x: busca_num_indice_selic(x.data),axis=1)

def corrige_selic(total,num_selic):
    # Número índice de referência (data da venda)
    num_indice_ref=df.loc[df.index[-1],'num_selic']
    total=total*num_indice_ref/num_selic
    return total
df['total_selic']=df.apply(lambda x: corrige_selic(x.total,x.num_selic),axis=1)

df

,parcela,saldo_devedor,itbi,custos_fixos,custos_iniciais,reforma,venda,total,data,num_selic,total_selic
0,0,"560,000.00","2,130.00",885.67,"154,000.00",0.00,0.00,"157,015.67",2022-09-22,100.53,"171,025.72"
1,"6,265.33","558,666.67","2,130.00",885.67,0.00,"12,500.00",0.00,"21,781.00",2022-10-22,101.60,"23,474.60"
2,"6,253.90","557,333.33","2,130.00",885.67,0.00,"12,500.00",0.00,"21,769.57",2022-11-22,102.72,"23,206.46"
3,"6,242.47","556,000.00","2,130.00",885.67,0.00,"12,500.00",0.00,"21,758.13",2022-12-22,103.81,"22,950.73"
4,"6,231.03","554,666.67","2,130.00",885.67,0.00,"12,500.00",0.00,"21,746.70",2023-01-22,104.95,"22,689.51"
5,"6,219.60","553,333.33","2,130.00",885.67,0.00,0.00,0.00,"9,235.27",2023-02-22,106.11,"9,530.31"
6,"6,208.17","552,000.00","2,130.00",885.67,0.00,0.00,0.00,"9,223.83",2023-03-22,107.16,"9,425.25"
7,"6,196.73","550,666.67","2,130.00",885.67,0.00,0.00,0.00,"9,212.40",2023-04-22,108.34,"9,311.04"
8,"6,185.30","549,333.33","2,130.00",885.67,0.00,0.00,"-847,100.00","-288,565.70",2023-05-22,109.50,"-288,565.70"


In [62]:
# calculo da IRR
def cal_irr(x):
    
    dates=list(x['data'])
    amounts=list(x['total'])
    t_aa=xirr(dates, amounts)
    t_am=(1+t_aa)**(1/12)-1
    print('TII a.a.(%): ',t_aa*100)
    print('TII a.m.(%): ',t_am*100)
    print('Lucro: ',-x['total'].sum())

    amounts_selic=list(x['total_selic'])
    t_aa=xirr(dates, amounts_selic)
    t_am=(1+t_aa)**(1/12)-1
    print('TII a.a. acima da selic (%): ',t_aa*100)
    print('TII a.m. acima da selic (%): ',t_am*100)
    print('Lucro: ',-x['total_selic'].sum())
    
    # return (t_aa*100,t_am*100,-x['total'].sum())

In [63]:
cal_irr(df)

TII a.a.(%):  11.60426689280194
TII a.m.(%):  0.9191072287721314
Lucro:  16823.13333333365
TII a.a. acima da selic (%):  -1.8958702253157682
TII a.m. acima da selic (%):  -0.15937887926649186
Lucro:  -3047.920756789972


In [64]:
valor_m2_aquisicao=valor_imovel/area
print(f'm2 aquisição: {valor_m2_aquisicao}')
valor_m2_venda=valor_venda/area
print(f'm2 reforma: {obra/area}')
print(f'm2 venda: {valor_m2_venda}')


m2 aquisição: 10215.827338129497
m2 reforma: 719.4244604316547
m2 venda: 12374.100719424461


In [65]:
11*66+70

796

In [66]:
### projeções de inflação e crescimento econômico para evolução dos preços de venda